In [ ]:
using JuMP, GLPK
using LinearAlgebra

include("loadSPP.jl")
include("setSPP.jl")
include("getfname.jl")


In [ ]:
# =========================================================================== #

# Loading a SPP instance
println("\nLoading...")
fname = "Data/didactic.dat"
C, A = loadSPP(fname)
@show C
@show A

# Solving a SPP instance with GLPK
println("\nSolving...")
solverSelected = GLPK.Optimizer
spp = setSPP(C, A)

set_optimizer(spp, solverSelected)
optimize!(spp)

# Displaying the results
println("z = ", objective_value(spp))
print("x = "); println(value.(spp[:x]))

# =========================================================================== #

# Collecting the names of instances to solve
println("\nCollecting...")
target = "Data"
fnames = getfname(target)

println("\nThat's all folks !")


In [ ]:
include("loadSPP.jl")
include("setSPP.jl")
include("getfname.jl")


println("\nLoading...")
fname = "Data/didactic.dat"
C, A = loadSPP(fname)
@show C
@show A

s = sum(A, dims=1)
C = reshape(C,  1, length(C))
println("C : ", C)
println("S : ", s)
v = C ./ s
sorted = sortperm(v, dims = 2, rev=true)
println("sort", v[sorted])


In [ ]:
println(typeof(A))

In [ ]:
function popfirst(array)
    item = array[1]
    array = array[2:end]
    return item
end

In [ ]:
function construct(C, A)
    s = sum(A, dims=1)
    C = reshape(C,  1, length(C))
    v = C ./ s
    sorted = sortperm(v, dims = 2)
    return vec(sorted) 
end

function optimiz(C, A)
    sorted = construct(C,A) #Construction Heuristic
    lenA, _ = size(A)
    temp = zeros(Int64, lenA)
    #temp = A[:, popped]
    res = zeros(length(C))
    best_res = C[sorted[1]]::Int64
    best_arr = zeros(length(C))
    best_arr[sorted[1]] = 1 #x0
    xbest, best_arr = multi_start(A, C, sorted, best_res, best_arr, res)
    return C[sorted[1]], xbest, best_arr
end

function multi_start(mat, cons, sorted, best_res, best_arr, res)
    sorted2 = sorted[:, 2:end]::Matrix{Int64}
    temp_best = 0
    for i in sorted:: Vector{Int64}
        res .*= 0
        res[i] = 1
        temp = mat[:, i]::Vector{Int64}
        sorted2 = sorted[:, 1:end .!= i]::Matrix{Int64}
        temp_best = cons[i]::Int64
        for j in sorted2::Matrix{Int64}
            adm, temp = rec_opt(mat, temp, j)
            if adm 
                res[j] = 1
                temp_best += cons[j]
                if temp_best > best_res
                    best_res = temp_best
                    best_arr = res[:]
                end 
            end
        end
    end
    return best_res, best_arr
end

function rec_opt(A::Matrix{Int64}, temp::Vector{Int64}, i::Int64)
    res = temp[:]::Vector{Int64}
    res .+= A[:, i]
    return rec_test(res, temp)
end 

function rec_test(res::Vector{Int64}, temp::Vector{Int64})
    if any(x -> x > 1, res)
        return false, temp
    else 
        return true, res
    end
end

In [ ]:
@time x0, xbest, res = optimiz(C, A)
#W = transpose(res) .* C
println("res : ", res)
println("x0 : ", x0)
println("xbest : ", xbest)
println("Constraints : ", C)
#println("Weights : ", W)
#println("Res Maximisation : ", sum(W))

In [ ]:
function eval(dir, nbRuns)
    files = getfname(dir)
    C, A = loadSPP(string(dir,files[1]))
    s = sum(A, dims=1)
    C = reshape(C,  1, length(C))
    v = C ./ s
    sorted = sortperm(v, dims = 2, rev=true)
    x0, res = optimiz(C, A)
    time_runs = zeros(length(files))
    for i in 1:length(files)
        C, A = loadSPP(string(dir,files[i]))
        s = sum(A, dims=1)
        C = reshape(C,  1, length(C))
        v = C ./ s
        sorted = sortperm(v, dims = 2, rev=true)
        temp_time = 0
        @time for j in 1:nbRuns
            start = time()
            x0, xbest, res = optimiz(C, A)
            temp_time += (time() - start)
        end
        time_runs[i] = temp_time / nbRuns
        println(time_runs[i])
        #W = transpose(res) .* C
    end
    println(time_runs)
end 

In [21]:
eval("./Data/", 10)